# Patronus GLIDER 모델 학습 및 평가 🚀

이 노트북은 [Patronus AI의 GLIDER GitHub 저장소](https://github.com/patronus-ai/glider)를 기반으로, Colab 환경에서 모델을 직접 **학습(Training)**하고 **평가(Evaluation)**하는 전체 과정을 안내합니다.

---

### 🚨 **시작 전 중요사항**

* **GPU 런타임 설정**: 이 코드는 GPU를 사용한 학습을 전제로 합니다. Colab 메뉴에서 **[런타임] > [런타임 유형 변경]**을 선택하고 하드웨어 가속기를 **GPU**로 설정하세요.
* **라이선스**: 이 코드는 `CC-BY-NC-4.0` 라이선스가 적용되어 **비상업적인(Non-Commercial)** 용도로만 사용해야 합니다.

In [6]:
# 1. GitHub 저장소 복제
!git clone https://github.com/patronus-ai/glider.git
%cd glider

# 2. 필요한 라이브러리 설치
# train_environment.yaml 파일의 pip 목록과 평가에 필요한 라이브러리를 모두 설치합니다.
!pip install accelerate==0.27.2 transformers==4.38.1 bitsandbytes==0.42.0 peft==0.9.0 trl==0.7.11 sentencepiece==0.1.99 wandb==0.16.3 datasets==2.18.0 einops==0.7.0 tyro==0.7.3
!pip install flash-attn --no-build-isolation
!pip install vllm scikit-learn

# 3. Accelerate 기본 설정
# Colab의 싱글 GPU 환경에 맞는 기본 설정을 자동으로 구성합니다.
!accelerate config default

print("\n✅ 환경 설정이 완료되었습니다.")

Cloning into 'glider'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 8), reused 13 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 20.03 KiB | 10.01 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/glider
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 M

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 52.6 MB/s eta 

Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 5, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 21, in <module>
    from accelerate.commands.estimate import estimate_command_parser
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/estimate.py", line 35, in <module>
    import timm
  File "/usr/local/lib/python3.12/dist-packages/timm/__init__.py", line 8, in <module>
    from .models import (
  File "/usr/local/lib/python3.12/dist-packages/timm/models/__init__.py", line 1, in <module>
    from .beit import *
  File "/usr/local/lib/python3.12/dist-packages/timm/models/beit.py", line 48, in <module>
    from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
  File "/usr/local/lib/python3.12/dist-packages/timm/data/__init__.py", line 5, in <module>
    from .dataset import ImageDataset, IterableImageDataset, AugMixDatas

### 📚 2단계 - 데이터셋 준비

모델 학습에는 **SFT 데이터셋**과 **DPO 선호도 데이터셋**이 필요합니다.
Google Drive를 마운트하고 각 데이터셋의 경로를 변수에 지정하세요.

**실행 전 확인:**
1.  사용할 데이터셋 파일들을 Google Drive에 업로드하세요.
2.  아래 코드 셀에서 `sft_dataset_path`와 `dpo_dataset_path` 변수의 경로를 실제 파일 경로로 수정하세요.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

# ⚠️ 아래 파일 경로를 자신의 Google Drive 파일 경로로 수정하세요.
sft_dataset_path = "/content/drive/MyDrive/path/to/your_sft_dataset.jsonl"
dpo_dataset_path = "/content/drive/MyDrive/path/to/your_dpo_dataset.jsonl"

# 모델 및 결과가 저장될 경로 설정
sft_model_output_path = "/content/glider/models/sft_model"
dpo_model_output_path = "/content/glider/models/final_glider_model"
evaluation_output_path = "/content/glider/outputs/evaluation_results"

print("SFT 데이터셋 경로:", sft_dataset_path)
print("DPO 데이터셋 경로:", dpo_dataset_path)

Mounted at /content/drive
SFT 데이터셋 경로: /content/drive/MyDrive/path/to/your_sft_dataset.jsonl
DPO 데이터셋 경로: /content/drive/MyDrive/path/to/your_dpo_dataset.jsonl


In [ ]:
# 1. SFT (Supervised Fine-Tuning) 학습 실행
# 모델이 평가의 기본 형식과 스타일을 학습합니다.
!python /content/glider/train/train_sft.py \
    --model_output_path="{sft_model_output_path}" \
    --dataset_path="{sft_dataset_path}"

# 2. DPO (Direct Preference Optimization) 학습 실행
# '좋은 답변'과 '나쁜 답변'을 구분하여 모델을 미세 조정합니다.
!python /content/glider/train/train_dpo.py \
    --model_dir="{sft_model_output_path}" \
    --model_output_path="{dpo_model_output_path}" \
    --dataset_path="{dpo_dataset_path}"

print("\n🧠 모델 학습이 완료되었습니다.")

In [ ]:
# 1. 학습된 모델로 예측 결과 생성
# eval 폴더로 이동하여 평가 스크립트를 실행합니다.
!python /content/glider/eval/vllm_test_local.py \
    --model="{dpo_model_output_path}" \
    --output_dir="{evaluation_output_path}"

# 2. 생성된 결과로부터 정확도 계산
!python /content/glider/eval/extract_accuracies.py \
    --eval_dir="{evaluation_output_path}" \
    --output_file="/content/glider/outputs/final_accuracy_results.jsonl"

print("\n📊 모델 평가가 완료되었습니다.")

In [ ]:
# 최종적으로 계산된 성능 평가 결과를 출력합니다.
print("--- 최종 성능 평가 결과 ---")
!cat /content/glider/outputs/final_accuracy_results.jsonl